In [1]:
#윤건웅 파일 병합
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

# 파일 경로 설정
d_yp1 = pd.read_csv('y_pm_1.csv')
d_yp2 = pd.read_csv('y_pm_2.csv')
d_yp3 = pd.read_csv('y_pm_3.csv')
d_yp4 = pd.read_csv('y_pm_4.csv')
d_yp5 = pd.read_csv('y_pm_5.csv')
d_yp6 = pd.read_csv('y_pm_6.csv')
d_ys1 = pd.read_csv('y_sub_1.csv')
d_ys2 = pd.read_csv('y_sub_2.csv')
d_ys3 = pd.read_csv('y_sub_3.csv')
d_ys4 = pd.read_csv('y_sub_4.csv')
d_ys5 = pd.read_csv('y_sub_5.csv')
d_ys6 = pd.read_csv('y_sub_6.csv')


In [2]:
file_paths_1 = ['y_pm_1.csv', 'y_pm_2.csv', 'y_pm_3.csv', 'y_pm_4.csv', 'y_pm_5.csv', 'y_pm_6.csv']
frames_1 = [pd.read_csv(file_path) for file_path in file_paths_1]
df_1 = pd.concat(frames_1, ignore_index=True)
df_1['대여건수'] = df_1['대여건수'].str.replace(',', '').astype(int)
df_1['반납건수'] = df_1['반납건수'].str.replace(',', '').astype(int)

file_paths_2 = ['y_sub_1.csv', 'y_sub_2.csv', 'y_sub_3.csv', 'y_sub_4.csv', 'y_sub_5.csv', 'y_sub_6.csv']
frames_2 = [pd.read_csv(file_path) for file_path in file_paths_2]
df_2 = pd.concat(frames_2, ignore_index=True)
df_2['승차승객수'] = df_2['승차승객수'].str.replace(',', '').astype(int)
df_2['하차승객수'] = df_2['하차승객수'].str.replace(',', '').astype(int)


In [3]:
app = Dash(__name__)

# 대시보드 레이아웃 구성
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='따릉이 대여소별 대여 및 반납 건수', children=[
            html.H1('따릉이 대여소별 대여 및 반납 건수'),
            dcc.Dropdown(
                id='month-dropdown-1',
                options=[
                    {'label': '2023년 1월', 'value': '202301'},
                    {'label': '2023년 2월', 'value': '202302'},
                    {'label': '2023년 3월', 'value': '202303'},
                    {'label': '2023년 4월', 'value': '202304'},
                    {'label': '2023년 5월', 'value': '202305'},
                    {'label': '2023년 6월', 'value': '202306'}
                ],
                value='202301',  # 기본값은 2023년 1월로 설정
                clearable=False
            ),
            dcc.Graph(id='rental-return-stats')
        ]),
        dcc.Tab(label='지하철 승하차 인원 수', children=[
            html.H1('지하철 승하차 인원 수'),
            dcc.Dropdown(
                id='month-dropdown-2',
                options=[
                    {'label': '2023년 1월', 'value': '202301'},
                    {'label': '2023년 2월', 'value': '202302'},
                    {'label': '2023년 3월', 'value': '202303'},
                    {'label': '2023년 4월', 'value': '202304'},
                    {'label': '2023년 5월', 'value': '202305'},
                    {'label': '2023년 6월', 'value': '202306'}
                ],
                value='202301',  # 기본값은 2023년 1월로 설정
                clearable=False
            ),
            dcc.Graph(id='passenger-data')
        ]),
    ])
])

# 콜백 함수 설정 - 대여건수와 반납건수
@app.callback(
    Output('rental-return-stats', 'figure'),
    [Input('month-dropdown-1', 'value')]
)
def update_graph_1(selected_month):
    selected_df = df_1[df_1['기준년월'] == int(selected_month)]  # 선택된 월 데이터만 사용

    rental_counts = selected_df.groupby('대여소명')['대여건수'].sum().reset_index()
    return_counts = selected_df.groupby('대여소명')['반납건수'].sum().reset_index()

    # 대여소명을 기준으로 데이터 정렬
    rental_counts = rental_counts.sort_values('대여소명')
    return_counts = return_counts.sort_values('대여소명')

    fig = go.Figure()
    fig.add_trace(go.Bar(x=rental_counts['대여소명'], y=rental_counts['대여건수'], name='대여건수', text=rental_counts['대여건수'], textposition='auto'))
    fig.add_trace(go.Bar(x=return_counts['대여소명'], y=return_counts['반납건수'], name='반납건수', text=return_counts['반납건수'], textposition='auto'))
    fig.update_layout(barmode='group', xaxis_tickangle=-45, yaxis=dict(title='건수', fixedrange=True),
                      title=f'{selected_month[:4]}년 {int(selected_month[4:])}월 대여 및 반납 건수', height=600)
    return fig
# 콜백 함수 설정 - 승하차 인원수 시각화
@app.callback(
    Output('passenger-data', 'figure'),
    [Input('month-dropdown-2', 'value')]
)
def update_graph_2(selected_month):
    selected_df = df_2[df_2['기준년월'] == int(selected_month)]  # 선택된 월 데이터만 사용

    fig = go.Figure()
    fig.add_trace(go.Bar(x=selected_df['역명'], y=selected_df['승차승객수'], name='승차승객수', text=selected_df['승차승객수'], textposition='auto'))
    fig.add_trace(go.Bar(x=selected_df['역명'], y=selected_df['하차승객수'], name='하차승객수', text=selected_df['하차승객수'], textposition='auto'))
    fig.update_layout(barmode='group', xaxis_tickangle=-45, yaxis=dict(title='인원수', fixedrange=True),
                      title=f'{selected_month[:4]}년 {int(selected_month[4:])}월 지하철 승하차 인원 수', height=600)
    return fig

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=8000)